## 라이브러리의 준비

In [ ]:
# 필요한 모듈의 인스톨
!pip install torch==1.7.0 torchvision==0.8.1
!pip install matplotlib==3.2.2

## 검증대상 데이터

In [ ]:
# 검증 샘플 화상
!curl https://raw.githubusercontent.com/jacobgil/pytorch-grad-cam/master/examples/both.png -o both.png

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

img = Image.open('both.png').convert('RGB')
plt.imshow(img)

In [ ]:
# 클래스 라벨 정보
!curl https://raw.githubusercontent.com/marcotcr/lime/master/doc/notebooks/data/imagenet_class_index.json -o imagenet_class_index.json

In [ ]:
import json

with open("imagenet_class_index.json", "r") as f:
    cls_idx = json.load(f)
    idx2label = [cls_idx[str(k)][1] for k in range(len(cls_idx))]

## AI모델의 준비와 예측

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
import torch.nn.functional as F

# 학습완료 모델을 읽어들임
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model.eval()
model.to(device)

In [ ]:
# 화상의 전처리
preprocess = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ) 
    ]
)

# 모델의 추론
img_tensor = preprocess(img).unsqueeze(0).to(device)
logits = model(img_tensor)
probs = F.softmax(logits, dim=1)

# 상위 5위의 결과를 확인
probs5 = probs.topk(5)
probability = probs5[0][0].detach().cpu().numpy()
class_id = probs5[1][0].detach().cpu().numpy()
for p, c in zip(probability, class_id):
    print((p, c, idx2label[c]))

## LIME에 의한 설명

In [ ]:
# LIME 인스톨
!pip install lime==0.2.0.1

### LIME에 의한 AI모델 설명

#### 화상의 영역 분할

In [ ]:
from lime.wrappers.scikit_image import SegmentationAlgorithm 
import numpy as np 
from skimage.segmentation import mark_boundaries 
 
# 화상의 영역분할（quickshift）
segmentation_fn = SegmentationAlgorithm( 
    'quickshift', 
    kernel_size=4, 
    max_dist=200, ratio=0.2, 
    random_seed=42 
) 
 
segments = segmentation_fn(img) 
plt.imshow(mark_boundaries(np.array(img), segments))

In [ ]:
# 화상의 영역분할（slic）
segmentation_fn = SegmentationAlgorithm("slic")
segments = segmentation_fn(img)
plt.imshow(mark_boundaries(np.array(img), segments))

#### LIME에 의한 설명 작성

In [ ]:
def batch_predict(images):
    # 화상의 전처리와 batch 화
    batch = torch.stack(tuple(preprocess(i) for i in images), dim=0)
    batch = batch.to(device)
    
    # 모델의 추론
    logits = model(batch)
    probs = F.softmax(logits, dim=1)
    return probs.detach().cpu().numpy()

from lime import lime_image

explainer = lime_image.LimeImageExplainer(random_state=42)
explanation = explainer.explain_instance(
    np.array(img), 
    batch_predict,
    top_labels=2, 
    hide_color=0,
    num_samples=5000,
    segmentation_fn=segmentation_fn
)

### LIME설명의 가시화와 해석

bull_mastiff의 가시화

In [ ]:
class_index = explanation.top_labels[0]
class_label = idx2label[class_index]
print(f"class_index: {class_index}, class_label: {class_label}")

In [ ]:
image, mask = explanation.get_image_and_mask(
    class_index, positive_only=False, num_features=5, hide_rest=False
)
img_boundry = mark_boundaries(image, mask)
plt.imshow(img_boundry)

tiger_cat의 가시화

In [ ]:
class_index = explanation.top_labels[1]
class_label = idx2label[class_index]
print(f"class_index: {class_index}, class_label: {class_label}")

In [ ]:
image, mask = explanation.get_image_and_mask(
    class_index,
    positive_only=False,
    negative_only=False,
    num_features=5,
    hide_rest=False
)
img_boundry = mark_boundaries(image, mask)
plt.imshow(img_boundry)

#### 기여도가 높은 부분 영역

bull_mastiff상위 5위의 영역과 기여도

In [ ]:
# bull_mastiff의 index 취득
index = explanation.top_labels[0]

for i in range(5):
    # 영역 Index와 기여도
    area_index, value = explanation.local_exp[index][i]
    print(f"area_index: {area_index}, value: {value}")

    # 화상의 가시화
    image = explanation.image.copy()
    c = 0 if value < 0 else 1
    image[segments == area_index, c] = np.max(image)
    plt.imshow(image) 
    plt.show()

tiger_cat의 상위 5위의 영역과 기여도

In [ ]:
# tiger_cat 인덱스의 취득
index = explanation.top_labels[1]

for i in range(5):
    # 영역의 index와 기여도
    area_index, value = explanation.local_exp[index][i]
    print(f"area_index: {area_index}, value: {value}")

    # 화상의 가시화
    image = explanation.image.copy()
    c = 0 if value < 0 else 1
    image[segments == area_index, c] = np.max(image)
    plt.imshow(image)
    plt.show()

#### 모든 영역에 대한 가시화

bull_mastiff에 대한 모든 영역의 가시화

In [ ]:
# bull_mastiff의 index 취득
index =  explanation.top_labels[0]

# heatmap의 생성
dict_heatmap = dict(explanation.local_exp[index])
heatmap = np.vectorize(dict_heatmap.get)(explanation.segments) 

# heatmap의 가시화
plt.imshow(img)
plt.imshow(heatmap, alpha=0.5, cmap='jet')
plt.colorbar()

tiger_cat에 대한 모든 영역의 가시화

In [ ]:
# tiger_cat의 index 취득
index =  explanation.top_labels[1]

# heatmap 생성
dict_heatmap = dict(explanation.local_exp[index])
heatmap = np.vectorize(dict_heatmap.get)(explanation.segments) 

# heatmap 가시화
plt.imshow(img)
plt.imshow(heatmap, alpha=0.5, cmap='jet')
plt.colorbar()

## Grad-CAM에 의한 설명

In [ ]:
# pytorch-grad-cam 의존 모듈 OpenCV 인스톨
!pip install opencv-python==4.5.1.48

In [ ]:
!git clone https://github.com/jacobgil/pytorch-grad-cam.git 
!cd pytorch-grad-cam && git checkout 6c83c8f  # Check out the latest commit hash on 2021/1

In [ ]:
# pytorch-grad-cam을 경로에 추가
import sys
sys.path.append("pytorch-grad-cam")

### Grad-CAM에 의한 AI모델 설명

In [ ]:
from gradcam import GradCam

grad_cam = GradCam(
    model=model,
    feature_module=model.layer4, 
    target_layer_names=["2"], 
    use_cuda=torch.cuda.is_available()
)

grayscale_cam = grad_cam(img_tensor, idx2label.index("bull_mastiff"))

### Grad-CAM의 설명 가시화와 해석

bull_mastiff에 대한 가시화

In [ ]:
import cv2

plt.imshow(img)
plt.imshow(
    cv2.resize(grayscale_cam, (image.shape[1], image.shape[0])),
    alpha=0.5,
    cmap='jet'
)
plt.colorbar()

tiger_cat에 대한 가시화

In [ ]:
grayscale_cam = grad_cam(img_tensor, idx2label.index("tiger_cat"))

plt.imshow(img)
plt.imshow(
    cv2.resize(grayscale_cam, (image.shape[1], image.shape[0])),
    alpha=0.5,
    cmap='jet'
)
plt.colorbar()